In [1]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

In [2]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","6g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 17:06:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

In [3]:
df_employee = spark.read.csv('employee.csv', header=True, inferSchema=True)
df_employee.printSchema()

[Stage 1:=================================================>         (5 + 1) / 6]

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: double (nullable = true)



In [4]:
df_employee = df_employee.withColumn("salary", df_employee["salary"].cast("decimal(10,2)"))
df_employee.printSchema()

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: decimal(10,2) (nullable = true)



In [5]:
%%time
df_employee.filter(df_employee["salary"] > 10000).count()

[Stage 2:=================================================>         (5 + 1) / 6]

CPU times: user 8.26 ms, sys: 9.02 ms, total: 17.3 ms
Wall time: 17.7 s


316067

In [6]:
# 14.2s do 17.5s

In [7]:
from pyspark.ml.feature import Bucketizer

# Zakładam, że kolumna "age" zawiera dane numeryczne. Jeśli nie, najpierw przekonwertujmy ją.
df_employee = df_employee.withColumn("age", df_employee["age"].cast("int"))

# Określamy zakres przedziałów wiekowych
splits = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# Tworzymy obiekt Bucketizer
bucketizer = Bucketizer(splits=splits, inputCol="age", outputCol="age_bucket")

# Tworzymy nowe kolumny z przydzielonymi bucketami
df_buckets = bucketizer.transform(df_employee)

# Wyświetlamy dane dla pierwszych 20 wierszy
df_buckets.select("age", "age_bucket").show(20)

# Grupowanie po bucketach i liczenie liczby osób w każdym z nich
df_buckets.groupBy("age_bucket").count().show()

+---+----------+
|age|age_bucket|
+---+----------+
| 67|       6.0|
| 61|       6.0|
| 50|       5.0|
| 48|       4.0|
| 44|       4.0|
| 27|       2.0|
| 29|       2.0|
| 32|       3.0|
| 37|       3.0|
| 26|       2.0|
| 63|       6.0|
| 67|       6.0|
| 30|       3.0|
| 39|       3.0|
| 34|       3.0|
| 57|       5.0|
| 43|       4.0|
| 44|       4.0|
| 36|       3.0|
| 27|       2.0|
+---+----------+
only showing top 20 rows



[Stage 6:=================================================>         (5 + 1) / 6]

+----------+-------+
|age_bucket|  count|
+----------+-------+
|       1.0| 785870|
|       4.0|3920172|
|       3.0|3918839|
|       2.0|3924223|
|       6.0|3529394|
|       5.0|3921502|
+----------+-------+

